This entire notebook is just a sketch of how the recursive algorithm ought to behave. It uses the ISL python bindings and polylib to construct the face lattice.

# Table of Contents

* Helper functions
* Actual algorithm
* Manual example

# Helper functions

In [1]:
from face_lattice import *   # this is my implementation of Loechner & Wilde's ScanFaces algorithm
from islpy import *
from functools import reduce

#### `inverse(op)`
Given an operation `op`, return its inverse. This is just hardcoded with '+' and 'max' for now.

In [2]:
def inverse(op):
    store = {
        '+': '-',
        'max': None,
        # ...
    }
    if op not in store:
        raise Exception('Operator "{}" not supported yet.'.format(op))
    return store[op]

#### `ker_from_map(f)`
Given an ISL map `f`, return the ISL set of points in the null space of `f`

In [3]:
def ker_from_map(f):
    if type(f) == str:
        f = BasicMap(f)
    mat = []
    for c in f.get_constraints():
        vars = c.get_var_dict()
        index_vals = [-1 * int(c.get_coefficient_val(v[0], v[1]).to_str()) for k, v in vars.items() if
                      v[0] != dim_type.param]
        mat.append(index_vals)
    mat = np.array(mat)
    
    indices = ','.join(['i{}'.format(i) for i in range(len(mat[0, :]))])
    constraints = []
    for c in mat:
        constraints.append(
            '+'.join(['{}{}'.format(a[0], a[1]) for a in zip(c, indices.split(','))]) + '=0')
    s = '{{[{}] : {}}}'.format(indices, ' and '.join(constraints))
    return BasicSet(s)

# examples
print('ex1: {}'.format(ker_from_map('{[i,j,k]->[i]}')))
print('ex2: {}'.format(ker_from_map('{[i,j,k]->[i,k]}')))
print('ex3: {}'.format(ker_from_map('{[i,j,k]->[i+k]}')))

ex1: { [i0, i1, i2] : i0 = 0 }
ex2: { [i0, i1, i2] : i0 = 0 and i2 = 0 }
ex3: { [i0, i1, i2] : i2 = -i0 }


#### `ker_from_facet_normal(facet, C)`
Given a facet `facet` (i.e., set of rows indices from `C` that are saturated), return the ISL set of points in the null space of the vector normal to the facet. 

For example, consider the $i=0$ face from the following set (3D cube),

$$ \{[i,j,k] : 0 \le i,j,k \le 100\} $$

The normal vector to the $i=0$ face is $[1,0,0]$ (and is equivalent to the vector representation of this constraint), therefore the null space is given by the set,

$$ \{[i,j,k] : i=0 \} $$

These are the points "in the facet".

In [4]:
def ker_from_facet_normal(facet, C):
    np_C = np.array(C)
    mat = np_C[np.array(list(facet)),:-1]
    indices = ','.join(['i{}'.format(i) for i in range(len(mat[0, :]))])
    constraints = []
    for c in mat:
        constraints.append('+'.join(['{}{}'.format(a[0], a[1]) for a in zip(c, indices.split(','))]) + '=0')
    s = '{{[{}] : {}}}'.format(indices, ' and '.join(constraints))
    return BasicSet(s)

#### `is_strict(facet, fp, C)`
Given a facet, represented by `facet` & `C`, and the projection function `fp`, determine whether the facet is considered a strict boundary. We'll call a facet a strict boundary if both of the following hold,

$$ ker(f_{p}) \cap ker(c) \ne \emptyset $$
$$ ker(f_{p}) \supseteq ker(c) $$

In [5]:
def is_strict(facet, fp, C):
    ker_c = ker_from_facet_normal(facet, C)
    ker_fp = ker_from_map(fp)
    return not ker_c.intersect(ker_fp).is_empty() and ker_fp.is_subset(ker_c)

#### `rho_from_labels(facets, C, labels, fd)`

Given a labeling (i.e., list of labels `labels`) and dependence function `fd`, return a $\rho$ that induces the labeling if possible. 
Each `facet` in `facets` is a set of indices of `C` that when saturated, describe the facet.
Each `label` in `labels` is either the string 'ADD', 'INV', or 'SUB' (if operator admits an inverse).

For example, if `facets = [{0,1},{0,2},{0,4}]` and `labels = ['ADD','ADD','INV']`, then the set that satifies the following constraints constitues the feasible space of valid $\rho$,

$$ \rho \cdot c_{0} > 0 $$
$$ \rho \cdot c_{1} > 0 $$
$$ \rho \cdot c_{2} = 0 $$

where $c_{i}$ denotes the normal vector of the $i$'th facet. Here $i=0$ corresponds to `facets[0]`, or constraints 0 and 1 from `C`. And $i=1$ denotes `facets[1]` or constraints 0 and 2. The conditions '>', '=', or '<' are chosen based on the labels 'ADD', 'INV', and 'SUB' respectively. Additionally, $\rho$ must be in the null space of the dependence function `fd`.

If this set is not trivially empty then we conclude that the labeling is valid. Any $\rho$ in the set is a valid $\rho$ that induces these labels, and we can just select the first choice (via lexmin or lexmax). Otherwise it returns None.

In [6]:
def rho_from_labels(facets, C, labels, fd):
    map = {'ADD': '>', 'INV': '=', 'SUB': '<'}
    bsets = list()

    # must be in ker(fd)
    bsets.append(ker_from_map(fd))

    for facet, label in zip(facets, labels):
        # select constraints representing this face, and create the set according to its label
        mat = C[np.array(list(facet)),:-1]
        indices = ','.join(['i{}'.format(i) for i in range(len(mat[0, :]))])
        constraints = []
        for c in mat:
            constraints.append(
                '+'.join(['{}{}'.format(a[0], a[1]) for a in zip(c, indices.split(','))]) + '{}0'.format(map[label]))
        s = '{{[{}] : {}}}'.format(indices, ' and '.join(constraints))
        bsets.append(BasicSet(s))

    # check that it's not trivially just the origin
    origin = BasicSet('{{[{}]}}'.format(('0,'*(C.shape[1]-1))[:-1]))
    result = reduce(lambda s0,s1: s0.intersect(s1), bsets)

    feasible_rho = result - origin

    if feasible_rho.is_empty():
        return None

    rho = list()
    # either lexmin or lexmax, not sure yet how to determine upfront which one to use, so for now just try both
    funcs = [feasible_rho.lexmin, feasible_rho.lexmax]
    for func in funcs:
        try:
            func().foreach_point(rho.append)
        except Exception:
            continue
        finally:
            if rho:
                break

    return rho[0]

#### `enumerate_labels(facets, labels)`
At each node in the face lattice, there are one or more facets that we potentially need to recurse into. This function just enumerates all possible labelings given the set of facets and the set of possible labels. If we have 4 facets and 3 possible labels, then there are $3^{4}$ combinations of labels.

In [7]:
# convert base-10 number n to base-d
def d_ary(d, n):
    if n == 0:
        return '0'
    nums = []
    while n:
        n, r = divmod(n, d)
        nums.append(str(r))
    return ''.join(reversed(nums))

LABEL = {
    0: 'ADD',
    1: 'INV',
    2: 'SUB'
}

def enumerate_labels(facets, labels):
    num_labels = len(labels)
    num_facets = len(facets)
    for i in range(num_labels ** num_facets):
        labels = [LABEL[int(d_ary(num_labels, i).zfill(num_facets)[j])] for j in range(num_facets)]
        yield labels

# example, all possible combos of 3 facets with 2 labels (8 combos total)
for labeling in enumerate_labels([{0},{1},{2}] , ['ADD','INV']):
    print(labeling)

['ADD', 'ADD', 'ADD']
['ADD', 'ADD', 'INV']
['ADD', 'INV', 'ADD']
['ADD', 'INV', 'INV']
['INV', 'ADD', 'ADD']
['INV', 'ADD', 'INV']
['INV', 'INV', 'ADD']
['INV', 'INV', 'INV']


# Recursive Algorithm

Initial setup given an input reduction expression $R$ with body $E$,

* Compute the context domain `s` of the reduction body
* Construct the face lattice `lattice` from `s`
* Initialize `fp` to the projection function of $R$
* Let `k` denote the remaining number of dimensions of reuse available, initialize to the rank of the null space of the depdence function `fd` present in the expression $E$ of the reduction body
* Initialize `node` to the root node in the face lattice
* Call `simplify(k, fp, fd, node, lattice, C, legal_labels)`, (see below for descriptions of `C` and `legal_labels`)

Define `simplify(k, fp, fd, node, lattice, C, legal_labels` where,
* `k`, `fp`, `node`, and `lattice` are initialized as described above
* `C` is the constraints matrix of the set `s`
* `legal_labels` is either ['ADD','INV','SUB'] or ['ADD','INV'] if the reduction operation does or does not admit an inverse, respectively

1. if `k == 0`, then return SUCCESS
1. For each facet of the current node in the lattice, label as either as 'strict' or 'weak' boundary given `fp`
1. Construct the list of candidate facets that potentially need to be recursed into, these are the non-strict facets
1. Enumerate all possible labelings and for each, construct the feasible space of legal reuse vectors $\rho$ (see `rho_from_labels` above). If the space for a particular labeling is empty, then conclude that the labeling is impossible. If at least one labeling is possible then continue, else return FAILURE.
1. For all possible labelings, recurse into each 'ADD' facet with $k-1$ (to indicate that there is one less dimension of reuse avilable. For a given labeling, if the recursion into all facets is successful then save the labeling as successful. As long as we check all possible labelings at a given label, then we don't need to backtrack.
1. If at least one labeling was successful then return SUCCESS and the set of all successful labelings else return FAILURE

In [8]:
def simplify(k=None, fp_str=None, fd_str=None, node=None, lattice=None, C=None, legal_labels=None):
    
    print('STEP 1 - if k==0 return else continue. k={}'.format(k))
    if k == 0:
        return True
    print()
    
    fp = BasicMap(fp_str)
    fd = BasicMap(fd_str)

    print('node = {}'.format(set(node)))
    print('fp = {}'.format(fp_str))
    print('fd = {}'.format(fd_str))
    print()

    print('STEP 2 - for each child face in node, label as "weak" or "strict" given fp\n')
    facets = list(lattice.graph.neighbors(node))
    for facet in facets:
        facet = set(facet)
        label = 'strict' if is_strict(facet, fp, C) else 'weak'
        print(facet, label)
    print()

    print('STEP 3 - construct candidate facets (i.e., non-strict facets)\n')
    candidate_facets = [facet for facet in facets if not is_strict(facet, fp, C)]
    print('candidate_facets = {}'.format([set(cf) for cf in candidate_facets]))
    print()

    print('STEP 4 - determine all possible combos\n')

    label_combos = list()
    header = ['{}'.format(set(f)) for f in candidate_facets]
    print(header)
    print('-' * len(str(header)))
    for labels in enumerate_labels(candidate_facets, legal_labels):
        rho = rho_from_labels(candidate_facets, C, labels, fd)
        if rho:
            label_combos.append(labels + [rho])
            print('{}  possible -> rho = {}'.format(labels, rho))
        else:
            print('{}  impossible'.format(labels))
    print()
    
    if not label_combos:
        print('FAILURE')
        return False

    print('STEP 4 - recurse into each "ADD" facet\n')
    print('todo - actually implement the recursive calls')

# Example

Take the following AlphaZ program (based on Firgure-2 from the 2006 paper, also the second example from the slides):

```
affine SR {N|}
input
	float X {i,j | 1<=i,j<=100};
output
	float Y {i | 1<=i<=100};
let
	Y[i] = reduce(max, (i,j,k->i), {| 1<=j<=i && 1<=k<=100-i } : (i,j,k->j,k)@X);
.
```

It has the following operator (op), projection function ($f_{p}$), body context domain (s) given be AlphaZ, and dependence function ($f_{d}$):

In [9]:
op='max'
fp='{[i,j,k]->[i]}'
s='{[i,j,k] : j>=1 and i>=j and k>=1 and 0>=i+k-100 and 0>=k-100 and 0>=j-100 and 0>=i-99 and i>=1}'
fd='{[i,j,k]->[j,k]}'

### Step 1 - construct the face lattice


In [10]:
C, lattice, bset, dim_P = face_lattice(s)

{[i,j,k] : j>=1 and i>=j and k>=1 and 0>=i+k-100 and 0>=k-100 and 0>=j-100 and 0>=i-99 and i>=1}

C_hat (constraints):
[[  1  -1   0   0]		{ [i, j, k] : i - j >= 0 }
 [  0   0   1  -1]		{ [i, j, k] : -1 + k >= 0 }
 [  0   1   0  -1]		{ [i, j, k] : -1 + j >= 0 }
 [ -1   0  -1 100]		{ [i, j, k] : 100 - i - k >= 0 }
 [  0   0   0   1]]		

R_hat (rays/vertices):
[[ 1 99  1 99]
 [ 1 99  1  1]
 [ 1  1 99  1]
 [ 1  1  1  1]]

Dimension:
3

3-faces: [{}]
2-faces: [{0}, {1}, {2}, {3}]
1-faces: [{0, 1}, {0, 2}, {0, 3}, {1, 2}, {1, 3}, {2, 3}]
0-faces: [{0, 1, 2}, {0, 1, 3}, {0, 2, 3}, {1, 2, 3}]



### Step 2
Determine the set of legal labels based on whether or not the operator admits an inverse

In [11]:
legal_labels = ['ADD', 'INV']
if inverse(op):
    legal_labels.append('SUB')
       
print('op = {}'.format(op))
print(legal_labels)

op = max
['ADD', 'INV']


### Step 3
Get the root node from the lattice

In [12]:
root = lattice.get_root()
print(set(root))

set()


### Step 4
Start the recursion at the root node

In [13]:
simplify(k=2, fp_str=fp, fd_str=fd, node=root, lattice=lattice, C=C, legal_labels=legal_labels)

STEP 1 - if k==0 return else continue. k=2

node = set()
fp = {[i,j,k]->[i]}
fd = {[i,j,k]->[j,k]}

STEP 2 - for each child face in node, label as "weak" or "strict" given fp

{0} weak
{1} weak
{2} weak
{3} weak

STEP 3 - construct candidate facets (i.e., non-strict facets)

candidate_facets = [{0}, {1}, {2}, {3}]

STEP 4 - determine all possible combos

['{0}', '{1}', '{2}', '{3}']
----------------------------
['ADD', 'ADD', 'ADD', 'ADD']  impossible
['ADD', 'ADD', 'ADD', 'INV']  impossible
['ADD', 'ADD', 'INV', 'ADD']  impossible
['ADD', 'ADD', 'INV', 'INV']  impossible
['ADD', 'INV', 'ADD', 'ADD']  impossible
['ADD', 'INV', 'ADD', 'INV']  impossible
['ADD', 'INV', 'INV', 'ADD']  impossible
['ADD', 'INV', 'INV', 'INV']  impossible
['INV', 'ADD', 'ADD', 'ADD']  impossible
['INV', 'ADD', 'ADD', 'INV']  impossible
['INV', 'ADD', 'INV', 'ADD']  impossible
['INV', 'ADD', 'INV', 'INV']  impossible
['INV', 'INV', 'ADD', 'ADD']  impossible
['INV', 'INV', 'ADD', 'INV']  impossible
['INV', 'IN

False

We see this doesn't work, because there are no possible labeling. If we decompose the reduction first, so that the projection function.

Choose $f = f_{1} \circ f_{2}$, where,

$$ f_{1} = \{[i,k] \rightarrow [i]\} $$
$$ f_{2} = \{[i,j,k] \rightarrow [i,k]\} $$

like this below, we see this works because what used to weak boundaries now become strict boundaries, and some of the strict boundaries can be ignored...

In [14]:
fp2 = '{[i,j,k]->[i,k]}'

In [15]:
simplify(k=2, fp_str=fp2, fd_str=fd, node=root, lattice=lattice, C=C, legal_labels=legal_labels)

STEP 1 - if k==0 return else continue. k=2

node = set()
fp = {[i,j,k]->[i,k]}
fd = {[i,j,k]->[j,k]}

STEP 2 - for each child face in node, label as "weak" or "strict" given fp

{0} weak
{1} strict
{2} weak
{3} strict

STEP 3 - construct candidate facets (i.e., non-strict facets)

candidate_facets = [{0}, {2}]

STEP 4 - determine all possible combos

['{0}', '{2}']
--------------
['ADD', 'ADD']  impossible
['ADD', 'INV']  possible -> rho = { [1, 0, 0] }
['INV', 'ADD']  impossible
['INV', 'INV']  impossible

STEP 4 - recurse into each "ADD" facet

todo - actually implement the recursive calls


What used to be weak boundary facets now become strict under the inner projection function $\{[i,j,k] \rightarrow [i,k]\}$. I still need to actually implement the recursive part, but in this case, I can manually just call simplify on the facet {0} for the only possible labeling.

It should look something like this below, though there are some issues with the below. It fails at this level when I would expect it not to. I'm still working through some things here.

In [16]:
node = list(lattice.graph.neighbors(root))[0]

simplify(k=1, fp_str=fp2, fd_str=fd, node=node, lattice=lattice, C=C, legal_labels=legal_labels)

STEP 1 - if k==0 return else continue. k=1

node = {0}
fp = {[i,j,k]->[i,k]}
fd = {[i,j,k]->[j,k]}

STEP 2 - for each child face in node, label as "weak" or "strict" given fp

{0, 1} weak
{0, 2} weak
{0, 3} weak

STEP 3 - construct candidate facets (i.e., non-strict facets)

candidate_facets = [{0, 1}, {0, 2}, {0, 3}]

STEP 4 - determine all possible combos

['{0, 1}', '{0, 2}', '{0, 3}']
------------------------------
['ADD', 'ADD', 'ADD']  impossible
['ADD', 'ADD', 'INV']  impossible
['ADD', 'INV', 'ADD']  impossible
['ADD', 'INV', 'INV']  impossible
['INV', 'ADD', 'ADD']  impossible
['INV', 'ADD', 'INV']  impossible
['INV', 'INV', 'ADD']  impossible
['INV', 'INV', 'INV']  impossible

FAILURE


False